In [91]:
#import packages
import numpy as np
import pandas as pd
import cv2
import time
import math
cv2.startWindowThread()


0

In [92]:
#open video
cap = cv2.VideoCapture('/Users/danielrosenthal/Documents/EGR_314/Daniel_7_top_trim.mp4')
fps = cap.get(cv2.CAP_PROP_FPS) #get fps

if cap.isOpened() == False:
    print("Cannot open video file.")

In [93]:
#saving output
frame_width = int(cap.get(3))
frame_height = int(cap.get(4))
size = (frame_width, frame_height)


In [94]:
#defining size for output videos
output = cv2.VideoWriter('output.mp4', cv2.VideoWriter_fourcc(*'mp4v'), 20.0, (1920,1080))
#output2 = cv2.VideoWriter('output2.mp4', cv2.VideoWriter_fourcc(*'MP4V'), 20.0, (1920,1080))

In [95]:
#object detection parameters
object_detector = cv2.createBackgroundSubtractorMOG2(history = 100, varThreshold= 90)

In [96]:
#creating empty array
# Create an empty DataFrame with columns for x and y velocities
velocities = pd.DataFrame(columns=['x_velocity', 'y_velocity'])
prev_x, prev_y = None, None
prev_time = None


In [97]:
#defining rgb range to detect orange for now

lower_color_bound = np.array([0,100,178])
upper_color_bound = np.array([255,255,255])

In [98]:
# #calculating ppi for full 1920x1080 frame
# pixels = frame_width*frame_height
# inches = 24*18
# ppi = math.sqrt(pixels) / math.sqrt(inches)

In [99]:
#conversion factor, likely incorrect bc camera and setup need to be calibrated
disc_length = 8.26 #disc length in inches
distance = 36 #distance of disc from camera, rough guess

#Approximate field of view (in degrees)
fov = 2 * np.arctan(frame_width / (2 * distance)) * 180 / np.pi
print(fov)

# Width of the object in pixels   
object_width_pixels = abs((disc_length/2) / (np.tan(fov/2)))
print (object_width_pixels)
#calculate conversion rate
conversion_rate = disc_length / object_width_pixels
print(conversion_rate)



175.70482914340297
36.661356887239556
0.22530535422912829


In [100]:
# disc_diameter = 8.26
# distance = 36
# disc_diameter_pixels = (disc_diameter * frame_height * frame_width) / (distance*2)


In [101]:
while True:
    # Get current time
    current_time = time.time()

    ret, frame = cap.read()
    if not ret:
        break
    #regions of interest
    height, width, _ = frame.shape
    roi = frame[200:900, 100:1800]
    
    #object detection
    mask = object_detector.apply(frame)

    #cleaning mask
    _, mask = cv2.threshold(mask, 254, 255, cv2.THRESH_BINARY)
    contours, _ = cv2.findContours(mask, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)

    #trying rgb threshold
    mask = cv2.cvtColor(frame,cv2.COLOR_BGR2HSV) 
    mask_rgb = cv2.inRange(mask,lower_color_bound,upper_color_bound)
    rgb_contours, _ = cv2.findContours(mask_rgb, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
       
    

    for cnt in rgb_contours:
        #calculate error, remove small elements
        area = cv2.contourArea(cnt)
        if area > 500:
            cv2.drawContours(frame, [cnt], -1, (0,255,0), 2)
            #drawing rectangle around detected objects
            x, y, w, h = cv2.boundingRect(cnt)
            rect_coordinates = np.append(rect_coordinates,[x,y,w,h])
            cv2.rectangle(frame, (x, y), (x + w, y + h), (0,255,0),3)
            
    #this is not working
    # disc_length_pixels = w
    # conversion_rate = (1/ppi) * (disc_length / disc_length_pixels)

    # Check if previous position and time are available
    if prev_x is not None and prev_y is not None and prev_time is not None:
        # Calculate the distance traveled in the x and y directions
        dx = x - prev_x
        dy = y - prev_y

        # Calculate the time interval
        dt = current_time - prev_time

        # Calculate the velocity in the x and y directions
        vx = (dx / dt) 
        vy = (dy / dt)
        
        # Add the velocity values to the DataFrame
        velocities = pd.concat([velocities, new_data])

        vx_mph = vx * conversion_rate * 0.44704
        vy_mph = vy * conversion_rate * 0.44704

        # Display the velocity values on the output video
        cv2.putText(frame, 'X Velocity: {:.2f} mph'.format(vx_mph), (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2)
        cv2.putText(frame, 'Y Velocity: {:.2f} mph'.format(vy_mph), (10, 60), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2)

        # create a new dataframe with the new velocities
        new_data = pd.DataFrame({'x_velocity': [vx_mph], 'y_velocity': [vy_mph]})

    # Update the previous position and time variables
    prev_x, prev_y = x, y
    prev_time = current_time
    
    cv2.imshow("frame", frame)
    #cv2.imshow("roi", roi)
    #cv2.imshow("Mask", mask)
    #cv2.imshow("maskrgb",mask_rgb)
    
    #output.write(roi)
    #output2.write(mask)
    if cv2.waitKey(120) & 0xFF == ord('q'):
        break


cap.release()
output.release()
#output2.release()
cv2.destroyAllWindows()

In [102]:
velocities.to_csv("velocities.csv", index=False)
print(velocities)

    x_velocity  y_velocity
0     0.000000    0.000000
0     0.000000    0.000000
0     0.000000    0.000000
0     0.000000    0.000000
0     0.000000    0.000000
0     0.000000    0.000000
0     0.000000    0.000000
0     0.000000    0.000000
0 -1130.151743 -130.961790
0     0.000000    5.178700
0     0.000000   15.433830
0     0.000000   19.682880
0    27.972874   19.459390
0    39.534070   18.531595
0    43.624746   15.396969
0    46.157691    6.154359
0    47.583701    1.853910
0    51.806048    0.647576
0    52.759744    1.286823
0    54.303453    1.939409
0    48.073375    0.000000
0    51.137126    0.616110
0    51.859575    0.000000
0    50.878470    1.211392
0    53.746883    1.279688
0    54.363254    1.264262
0    53.666068    0.000000
0    52.526645    1.250634
0    54.182195    1.231414
0    56.349273    0.640333
0    54.616339    0.620640
0    48.777762    1.121328
0    55.040600    1.265301
0    53.689701    1.248598
0    53.740368   15.877836
0     0.000000    0.000000
0